In [1]:
%matplotlib inline 
import numpy as np

import arrayfire as af
from scipy import ndimage
import matplotlib.pyplot as plt
from LearnAF import *
import tqdm
import pandas as pd

af.set_backend('cpu')
#af.set_device(1)

f = np.load('/home/narsi/Downloads/mnist.npz')
Data = f['x_train']
y_train = f['y_train']

Data = Data.reshape(60000, 784)
Data = Data.astype('float32')/255.0
classes = np.asarray(to_categorical(y_train), dtype = np.float32)

# initialize weights randomly with mean 0
syn0 = np.array(2*np.random.random((Data.shape[1],64)) - 1, dtype = np.float32)
W1 = Variable(af.np_to_af_array(syn0),name='W1')
b1 = Variable(af.constant(0,1),name='b1')

syn0 = np.array(2*np.random.random((64,16)) - 1, dtype = np.float32)
W2 = Variable(af.np_to_af_array(syn0),name='W2')
b2 = Variable(af.constant(0,1),name='b2')

syn0 = np.array(2*np.random.random((16,10)) - 1, dtype = np.float32)
W3 = Variable(af.np_to_af_array(syn0),name='W3')
b3 = Variable(af.constant(0,1),name='b3')

w = [W1,b1,W2,b2,W3,b3]

# Model

In [2]:
def learner(X,W):
    
    # 784 -> 64
    X1 = relu(add(matmul(X,W[0]),W[1]))
    # 64 -> 16
    X2 = relu(add(matmul(X1,W[2]),W[3]))
    # 16 -> 10
    YP = softmax(add(matmul(X2,W[4]),W[5]))
    return YP

# Place Holders

In [3]:
batch = 64
Xin = Constant(af.np_to_af_array(np.random.random((batch,Data.shape[1])).astype(np.float32)))
Y = Constant(af.np_to_af_array(np.random.random((batch,10)).astype(np.float32)))

In [4]:
YP = learner(Xin,w)
e = CrossEntropy(Y,YP)
acc = accuracy(Y,YP)
sgd = SGD(lr = 0.01,momentum=0.9)

In [5]:
for i in range(1):
    epoch_acc = []
    epoch_loss = []
    total_batchs = int(Data.shape[0]/batch)
    for j in tqdm.tqdm(range(total_batchs)):
        X_np = Data[j*batch:(j+1)*batch,:]
        Xin.value = af.np_to_af_array(X_np)
        Y_np = classes[j*batch:(j+1)*batch,:]
        Y.value = af.np_to_af_array(Y_np)
        
        (l,w) = sgd.update(e, w, i)
        acc = accuracy(Y,YP)
        epoch_acc.append(acc)
        epoch_loss.append(np.asarray(l)[0])
    print('Accuracy :'+str(np.mean(epoch_acc)))
    print('Loss :'+str(np.mean(epoch_loss)))

100%|██████████| 937/937 [00:28<00:00, 33.09it/s]

Accuracy :0.101921024546
Loss :968.812
